In [24]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [26]:
df = pd.read_csv(r"C:\MY FILE\MBAN 6110S-Data Science\module_5_feature_engineering_demo.csv")

In [27]:
df.head()


,Unnamed: 0,Height,Weight,Calories_Intake,Exercise_Time,BMI,Age,Resting_Heart_Rate
0,0,187.640523,81.119254,1733.539473,4.593274,23.039374,70,54.540176
1,1,174.001572,87.849478,1644.014918,3.568722,29.015686,38,63.510746
2,2,179.787380,61.553704,2523.067529,2.885513,19.043018,76,49.502603
3,3,192.408932,72.094281,2020.812760,3.251630,19.473784,35,72.310220
4,4,188.675580,74.561067,2459.594194,1.789144,20.945017,49,67.864721


In [28]:
df.columns

Index(['Unnamed: 0', 'Height', 'Weight', 'Calories_Intake', 'Exercise_Time',
       'BMI', 'Age', 'Resting_Heart_Rate'],
      dtype='object')

In [29]:
df.drop('Unnamed: 0', axis = 1, inplace=True)

In [48]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# standardize the feature 缩放器
scaler = StandardScaler() # Z-score 标准化， 将特征值转换为0-1的分布，消除偏差
df_std = scaler.fit_transform(df) #对df的数据 进行标准化处理
       #  or scaler.fit(df)
# Perform PCA
pca = PCA() #    n_components默认为none, 表示保留所有的主成分
df_pca = pca.fit_transform(df_std) # 转换后的数据与原始数据相同的维度


# The transformed data is an array, convert it back to a dataframe
df_pca = pd.DataFrame(df_pca, columns=[f'PC{i+1}' for i in range(len(df.columns))])

# Print the explained variance ratio
print('Explained variance: ', pca.explained_variance_ratio_)

# print the cumulative explained variance ratio
cumsum_variance = np.cumsum(pca.explained_variance_ratio_)
print('Cumulative explained variance ratio: ', cumsum_variance)

#show the first few rows of transfomed dataframe
df_pca.head()




Explained variance:  [0.28857346 0.18072185 0.14675488 0.14381463 0.13558216 0.10353415
 0.00101888]
Cumulative explained variance ratio:  [0.28857346 0.46929531 0.61605019 0.75986482 0.89544697 0.99898112
 1.        ]


,PC1,PC2,PC3,PC4,PC5,PC6,PC7
0,-0.395268,-1.182948,1.791319,2.336967,0.037477,0.366259,-0.018202
1,0.819082,-0.205962,2.084828,0.028116,-0.058311,0.446441,-0.061576
2,-1.171866,-0.469183,-0.324207,1.639464,-0.240676,-0.935364,0.040073
3,-1.106793,0.798769,2.060352,0.535101,1.421538,0.537776,0.082936
4,-0.772500,1.500282,0.797973,0.753864,1.115887,-0.730499,0.036682


In [47]:
# let's do the same, but now reduce to 2 components 
# perform PCA
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_std)

# The transormed data is an array, convert it back into a dataframe 
df_pca = pd.DataFrame(df_pca)

# Print the explained variance ration
print('Explained variance ratio: ', pca.explained_variance_ratio_)

# Print the cumulative explained variance ratio 
cumsum_variance = np.cumsum(pca.explained_variance_ratio_)
print('Cumulative explained variance ratio: ', cumsum_variance)

Explained variance ratio:  [0.28857346 0.18072185]
Cumulative explained variance ratio:  [0.28857346 0.46929531]


we can see that with 2 components, only 46% of the variability in data is explained. 

In [49]:
df2 = pd.read_csv(r"C:\MY FILE\MBAN 6110S-Data Science\bikes_sharing.csv")

In [51]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [76]:
# Transform datetime to a datetime data format
df2['datetime'] = pd.to_datetime(df2['datetime'])
df2['hour'] = df2['datetime'].dt.hour
df2['month'] = df2['datetime'].dt.month
df2['day'] = df2['datetime'].dt.day

# create funciton to classify amounts 
def total_cat(x):
    if x >= 0 and x < 10:
        return '0-10'
    elif x >= 50 and x <100:
        return '50-100'
    else:
        return '100'
    
# create new column with rental count range using total_cat()function
df2['rental_total_group'] = df2['count'].apply(total_cat)

# create function with 2 inputs -temp and humidity to classify 
def good_bad(temp,hum):
    if temp > 24 and hum >70:
        return 'too hot'
    elif temp <=25 and hum >=50 and hum <=70:
        return 'so so day'
    else:
        return 'good day'
    
# apply function
df2['day_type'] = df2.apply(lambda x: good_bad(x['temp'], x['humidity']), axis=1)

# Dummy variables -convert season to dummies
# first, rename season
season_mapping ={1:'winter', 2:'spring', 3:'summer', 4:'fall'}
df2['season'] = df2['season'].map(season_mapping)

# Create season dummies
season_dummies = pd.get_dummies(df2['season'])
season_dummies.head()

df2 = pd.concat([df2,season_dummies], axis =1)
